In [1]:
import numpy as np
import pickle

padded_features = np.load("../../data/processed_data/rtp_features_padded.npy")
padded_target = np.load("../../data/processed_data/target_padded.npy")

with open("../../data/processed_data/alphabet", "rb") as f:
    alphabet = pickle.load(f)

print(padded_features.shape, padded_target.shape)

(11983, 1713, 4) (11983, 64)


In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [7]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

TensorFlow version: 2.9.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [8]:
def split_data(features, target, train_size=0.9):
    size = len(features)
    indices = np.arange(size)
    np.random.shuffle(indices)
    train_samples = int(size * train_size)
    
    x_train, y_train = features[indices[:train_samples]], target[indices[:train_samples]]
    x_valid, y_valid = features[indices[train_samples:]], target[indices[train_samples:]]
    
    return (
        x_train,
        x_valid,
        y_train,
        y_valid,
    )

x_train, x_valid, y_train, y_valid = split_data(test_features, padded_target)

In [9]:
print(len(x_train), len(x_train[0]), len(x_train[0][0]))

10784 1713 4


In [10]:
OUTPUT_SIZE = len(alphabet) + 1

N_BLSTM_LAYERS = 5
N_CELLS = 64

LEARNING_RATE = 10**-3

BATCH_SIZE = 8
N_FEATURES = len(x_train[0][0])
N_TIMESTEPS = 1713
N_EPOCHS = 1

In [11]:
from keras.layers import Bidirectional, LSTM, Dense, Input, Masking, TimeDistributed

def build_model(n_blstm_layers, n_cells, n_features, output_size):
    model = tf.keras.models.Sequential()

    model.add(Masking(input_shape=(None, N_FEATURES), mask_value=np.zeros((N_FEATURES))))
    
    for i in range(n_blstm_layers):
        model.add(Bidirectional(LSTM(N_CELLS,
                                     input_shape=(None, N_FEATURES),
                                     return_sequences = True,
                                     dropout = 0.5),
                                merge_mode = 'sum'))

    model.add(
        TimeDistributed(
            Dense(output_size, activation = 'softmax')
        )
    )

    return model

In [12]:
def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss

In [13]:
model = build_model(N_BLSTM_LAYERS, N_CELLS, N_FEATURES, OUTPUT_SIZE)

model.compile(
    loss=ctc_loss,
    optimizer= tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, clipnorm=9)
)

In [14]:
def create_dataset(train_data, valid_data, n_epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((train_data, valid_data))
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)
    return dataset

train_dataset = create_dataset(x_train, y_train, N_EPOCHS, BATCH_SIZE)
validation_dataset = create_dataset(x_valid, y_valid, N_EPOCHS, BATCH_SIZE)

In [15]:
from pathlib import Path
loss_history = []

if Path("../../training/encoder/loss").is_file():
    with open("../../training/encoder/loss", "rb") as f:
        loss_history = pickle.load(f)

In [ ]:
# %%capture stored_output

import keras

checkpoint_path = "../../training/encoder/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

if Path(checkpoint_path).is_file():
    model.load_weights(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

logdir = '../../training/encoder/logs'
tb_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq=1)

history = model.fit(
    train_dataset,
    validation_data = validation_dataset,
    shuffle=True,
    epochs = N_EPOCHS,
    batch_size = BATCH_SIZE,
    callbacks = [cp_callback, tb_callback]
)

1348/1348 [==============================] - ETA: 0s - loss: 193.3737

In [ ]:
stored_output.show()

In [ ]:
loss_history.append([])

In [ ]:
with open("../../training/encoder/loss", "wb") as f:
    pickle.dump(loss_history, f)

In [ ]:
print(total_history)

In [ ]:
import matplotlib.pyplot as plt 

loss = []
val_loss = []

for training_session in total_history:
    print(training_session.history)
    loss.append(training_session.history['loss'])
    val_loss.append(training_session.history['val_loss'])

    
def flatten_list(l):
     return [point for elem in l for point in elem]

plt.plot(flatten_list(loss), color="red", label="train")
plt.plot(flatten_list(val_loss), color="black", label="test")
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
train:[100.6691, 95.1948]
valid:[97.5810, 91.9827]